In [1]:
import os
import sys
import json
import torch

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')
from fgc_support_retri.utils import *
from fgc_support_retri import config

In [3]:
data = json_load(config.FGC_TRAIN)

In [4]:
def get_fold_datas(data, fold):
    fold_len = int(len(data)/fold)
    fold_datas = []
    for i in range(fold):
        if i == fold-1:
            fold_data = data[fold_len*i:]
        else:
            fold_data = data[fold_len*i:fold_len * (i+1)]
        fold_datas.append(fold_data)
    return fold_datas

In [5]:
fold_datas = get_fold_datas(data, 10)

In [6]:
len(fold_datas[0])

88

In [7]:
from fgc_support_retri.train import *

def train_entity_model(num_epochs, batch_size, model_file_name, train_documents, lr):
    dataset_reader = SerSentenceDataset
    
    tokenizer = BertTokenizer.from_pretrained(bert_model_name)
    pretrained_bert = BertModel.from_pretrained(bert_model_name)
    pretrained_bert.eval()
    
    model = EntitySERModel.from_pretrained(bert_model_name)
    model.to_mode('etype+idf')
    
    collate_fn = Sent_collate
    indexer = SentIdx(tokenizer, pretrained_bert)
    input_names = ['input_ids', 'question_ids', 'token_type_ids', 'attention_mask', 
                   'tf_type', 'idf_type', 'sf_type', 'sf_score', 'qsim_type', 'atype_label', 'etype_ids', 'label']
    trainer = SER_Trainer(model, collate_fn, indexer, dataset_reader, input_names, lr)
    trainer.train(num_epochs, batch_size, model_file_name, train_documents)

I0325 07:31:16.839226 140363791722304 file_utils.py:39] PyTorch version 1.1.0 available.


In [ ]:
for i, predict_document in enumerate(fold_datas):
    train_documents = []
    for j, fold_data in enumerate(fold_datas):
        if j != i:
            train_documents += fold_data
    train_entity_model(10, 12, "fold_{}_entity".format(i), train_documents, 2e-5)
    torch.cuda.empty_cache()

I0325 07:31:18.984321 140363791722304 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0325 07:31:19.929808 140363791722304 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0325 07:31:19.932777 140363791722304 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

dev_set indexing...


  0%|          | 0/794 [00:00<?, ?it/s]

train_set indexing...
{'text': '電視名人。為川普集團前任董事長兼總裁', 'text_cn': '电视名人。为川普集团前任董事长兼总裁', 'start': 76, 'end': 94}
{'text': '川普娛樂公司的創辦人，在全世界經營房地產', 'text_cn': '川普娱乐公司的创办人，在全世界经营房地产', 'start': 95, 'end': 115}
{'text': '熱，電，輻射', 'text_cn': '热，电，辐射', 'start': 22, 'end': 28}
{'text': '', 'text_cn': '', 'start': 0, 'end': 0}
{'text': '校園注意教室之空氣流通，落實勤洗手，有呼吸道症狀時應配戴口罩；打噴嚏時應用面紙', 'text_cn': '校园注意教室之空气流通，落实勤洗手，有呼吸道症狀时应配戴口罩；打喷嚏时应用面纸', 'start': 88, 'end': 127}
{'text': '他人交談時，儘可能保持1公尺以上之距離，避免病毒傳播。', 'text_cn': '他人交谈时，尽可能保持1公尺以上之距离，避免病毒传播。', 'start': 143, 'end': 170}
{'text': '', 'text_cn': '', 'start': 0, 'end': 0}


  0%|          | 0/2423 [00:00<?, ?it/s]

...

  1%|          | 2/239 [00:00<00:13, 18.04it/s]

epoch 7 train_loss: 0.011


100%|██████████| 239/239 [00:19<00:00, 12.57it/s]


{'sp_em': 0.159, 'sp_prec': 0.593, 'sp_recall': 0.592, 'sp_f1': 0.54}
epoch 7 eval_recall: 0.592 eval_f1: 0.540


 39%|███▉      | 953/2423 [03:06<04:43,  5.19it/s]